In [1]:
import sys
import os
from pathlib import Path

root = Path(os.getcwd()).parent.parent
sys.path.append(str(Path(os.getcwd()).parent))

In [2]:
from constant import FeatureType
from loader import Power, Weather
from model import Model

import argparse

### Experiment

In [3]:
parser = argparse.ArgumentParser()
args = parser.parse_args("")

# ====== Path ====== #
args.root = root
args.experiment_name = "LSTM-001"
args.automl_name = 'automl-001'

# ====== Model ====== #
args.frame_in = 72
args.frame_out = 24
args.epochs = 200

# ====== Data ====== #
args.years = [2017, 2018, 2019]
args.region = "Jindo"
args.station = 192
args.ratio = [0.6, 0.2, 0.2]

# ====== Features ====== #
features = [FeatureType.SUNSHINE,
            FeatureType.GROUND_TEMPERATURE,
            FeatureType.HUMIDITY,
            FeatureType.WIND_SPEED,
            FeatureType.WIND_DIRECTION,
            FeatureType.TEMPERATURE,
            FeatureType.VISIBILITY,
            FeatureType.PRECIPITATION,
            FeatureType.STEAM_PRESSURE,
            FeatureType.DEW_POINT_TEMPERATURE,
            FeatureType.ATMOSPHERIC_PRESSURE]

power = Power(args)
weather = Weather(args, features)

power_data = power.get_data()
model_manager = Model(power_data['scaler'])
setattr(args, 'test_start', power.test_start)
setattr(args, 'test_end', power.test_end)

y_pred_list = []
results = []
for i in range(len(features)):
    weather_data = weather.get_data(i + 1)
    setattr(args, 'feature_len', i + 1)
    setattr(args, 'name', 'feature-%02d' % (i + 1))

    train = [weather_data['train'], power_data['train']]
    val = [weather_data['val'], power_data['val']]
    test = [weather_data['test'], power_data['test']]

    dataset = {'train': train, 'val': val, 'test': test}
    args = model_manager.read_setting(args)
    model = model_manager.train(args, dataset)
    y_pred = model.predict(dataset['test'][0])
    y_pred_list.append(y_pred)
    result = model_manager.evaluate(model, dataset['test'][0], dataset['test'][1])
    results.append(result)

setattr(args, 'features', features)
model_manager.save_result(dataset['test'][1], y_pred_list, args)

train start date: 2017-01-01 00:00:00
train end date: 2018-10-19 23:00:00
val start date: 2018-10-20 00:00:00
val end date: 2019-05-26 23:00:00
test start date: 2019-05-27 00:00:00
test end date: 2019-12-31 23:00:00
train start date: 2017-01-01 00:00:00
train end date: 2018-10-19 23:00:00
val start date: 2018-10-20 00:00:00
val end date: 2019-05-26 23:00:00
test start date: 2019-05-27 00:00:00
test end date: 2019-12-31 23:00:00
0 missing dates
0 value(s) are not zero
0 missing dates
0 value(s) are not zero
0 missing dates
0 value(s) are not zero
(15768,)
(5256,)
(5256,)
missing dates: []
missing dates: ['2018-01-13 09:00', '2018-01-13 10:00', '2018-01-13 11:00', '2018-01-13 12:00', '2018-01-13 13:00']
missing dates: []
Epoch 1/200
5/6 [========================>.....] - ETA: 0s - loss: 0.0024
Epoch 00001: val_loss improved from inf to 0.00184, saving model to /home/vislab/Github/SolarPowerPrediction/results/LSTM-001/feature-01/model-001-0.001842.h5
6/6 [==============================] -

6/6 [==============================] - 0s 51ms/step - loss: 0.0015 - val_loss: 0.0013
Epoch 26/200
6/6 [==============================] - ETA: 0s - loss: 0.0015
Epoch 00026: val_loss improved from 0.00131 to 0.00130, saving model to /home/vislab/Github/SolarPowerPrediction/results/LSTM-001/feature-01/model-026-0.001300.h5
6/6 [==============================] - 0s 21ms/step - loss: 0.0015 - val_loss: 0.0013
Epoch 27/200
5/6 [========================>.....] - ETA: 0s - loss: 0.0015
Epoch 00027: val_loss improved from 0.00130 to 0.00129, saving model to /home/vislab/Github/SolarPowerPrediction/results/LSTM-001/feature-01/model-027-0.001293.h5
6/6 [==============================] - 0s 47ms/step - loss: 0.0015 - val_loss: 0.0013
Epoch 28/200
5/6 [========================>.....] - ETA: 0s - loss: 0.0015
Epoch 00028: val_loss improved from 0.00129 to 0.00129, saving model to /home/vislab/Github/SolarPowerPrediction/results/LSTM-001/feature-01/model-028-0.001291.h5
6/6 [=======================

6/6 [==============================] - 0s 18ms/step - loss: 0.0011 - val_loss: 9.7444e-04
Epoch 52/200
5/6 [========================>.....] - ETA: 0s - loss: 0.0011
Epoch 00052: val_loss improved from 0.00097 to 0.00095, saving model to /home/vislab/Github/SolarPowerPrediction/results/LSTM-001/feature-01/model-052-0.000948.h5
6/6 [==============================] - 0s 51ms/step - loss: 0.0011 - val_loss: 9.4824e-04
Epoch 53/200
5/6 [========================>.....] - ETA: 0s - loss: 0.0011
Epoch 00053: val_loss improved from 0.00095 to 0.00094, saving model to /home/vislab/Github/SolarPowerPrediction/results/LSTM-001/feature-01/model-053-0.000937.h5
6/6 [==============================] - 0s 23ms/step - loss: 0.0011 - val_loss: 9.3695e-04
Epoch 54/200
5/6 [========================>.....] - ETA: 0s - loss: 0.0011
Epoch 00054: val_loss improved from 0.00094 to 0.00094, saving model to /home/vislab/Github/SolarPowerPrediction/results/LSTM-001/feature-01/model-054-0.000935.h5
6/6 [===========

6/6 [==============================] - ETA: 0s - loss: 7.2546e-04
Epoch 00077: val_loss improved from 0.00063 to 0.00063, saving model to /home/vislab/Github/SolarPowerPrediction/results/LSTM-001/feature-01/model-077-0.000630.h5
6/6 [==============================] - 0s 26ms/step - loss: 7.2546e-04 - val_loss: 6.2985e-04
Epoch 78/200
5/6 [========================>.....] - ETA: 0s - loss: 7.1676e-04
Epoch 00078: val_loss improved from 0.00063 to 0.00062, saving model to /home/vislab/Github/SolarPowerPrediction/results/LSTM-001/feature-01/model-078-0.000617.h5
6/6 [==============================] - 0s 26ms/step - loss: 7.1934e-04 - val_loss: 6.1709e-04
Epoch 79/200
6/6 [==============================] - ETA: 0s - loss: 7.1299e-04
Epoch 00079: val_loss did not improve from 0.00062
6/6 [==============================] - 0s 41ms/step - loss: 7.1299e-04 - val_loss: 6.3972e-04
Epoch 80/200
5/6 [========================>.....] - ETA: 0s - loss: 7.1122e-04
Epoch 00080: val_loss did not improve 

Epoch 106/200
6/6 [==============================] - ETA: 0s - loss: 6.3266e-04
Epoch 00106: val_loss did not improve from 0.00054
6/6 [==============================] - 0s 13ms/step - loss: 6.3266e-04 - val_loss: 5.4901e-04
Epoch 107/200
6/6 [==============================] - ETA: 0s - loss: 6.3107e-04
Epoch 00107: val_loss did not improve from 0.00054
6/6 [==============================] - 0s 13ms/step - loss: 6.3107e-04 - val_loss: 5.8328e-04
Epoch 108/200
1/6 [====>.........................] - ETA: 0s - loss: 5.9391e-04
Epoch 00108: val_loss improved from 0.00054 to 0.00054, saving model to /home/vislab/Github/SolarPowerPrediction/results/LSTM-001/feature-01/model-108-0.000536.h5
6/6 [==============================] - 0s 16ms/step - loss: 6.3071e-04 - val_loss: 5.3556e-04
Epoch 109/200
5/6 [========================>.....] - ETA: 0s - loss: 6.2714e-04
Epoch 00109: val_loss improved from 0.00054 to 0.00053, saving model to /home/vislab/Github/SolarPowerPrediction/results/LSTM-001/fea

...

Epoch 48/200
 8/11 [====================>.........] - ETA: 0s - loss: 4.1840e-04
Epoch 00048: val_loss did not improve from 0.00032
11/11 [==============================] - 0s 10ms/step - loss: 4.9001e-04 - val_loss: 3.4208e-04
train_score
in test datsaet, zeros: 7154, non_zeros: 8542
in prediction, zeros: 0, non_zeros: 15696
true_positive: 0, true_negative: 8543, false_positive: 0, false_negative: 7154
nrmse: 0.027176, accuracy: 0.544244, f1_score: -1.000000
{'nrmse': 0.027175832787056427, 'accuracy': 0.5442441230808435, 'f1_score': -1}
val_score
in test datsaet, zeros: 2547, non_zeros: 2637
in prediction, zeros: 0, non_zeros: 5184
true_positive: 0, true_negative: 2638, false_positive: 0, false_negative: 2547
nrmse: 0.158862, accuracy: 0.508775, f1_score: -1.000000
{'nrmse': 0.15886197028947704, 'accuracy': 0.5087753134040501, 'f1_score': -1}
test_score
in test datsaet, zeros: 2365, non_zeros: 2819
in prediction, zeros: 0, non_zeros: 5184
true_positive: 0, true_negative: 2820, false_p

Epoch 22/200
324/327 [============================>.] - ETA: 0s - loss: 3.7077e-04
Epoch 00022: val_loss did not improve from 0.00043
327/327 [==============================] - 2s 5ms/step - loss: 3.6987e-04 - val_loss: 4.3816e-04
Epoch 23/200
326/327 [============================>.] - ETA: 0s - loss: 3.6883e-04
Epoch 00023: val_loss did not improve from 0.00043
327/327 [==============================] - 2s 5ms/step - loss: 3.6848e-04 - val_loss: 4.4512e-04
Epoch 24/200
323/327 [============================>.] - ETA: 0s - loss: 3.6780e-04
Epoch 00024: val_loss did not improve from 0.00043
327/327 [==============================] - 2s 5ms/step - loss: 3.6603e-04 - val_loss: 4.4263e-04
Epoch 25/200
323/327 [============================>.] - ETA: 0s - loss: 3.6633e-04
Epoch 00025: val_loss did not improve from 0.00043
327/327 [==============================] - 2s 5ms/step - loss: 3.6455e-04 - val_loss: 4.4594e-04
Epoch 26/200
324/327 [============================>.] - ETA: 0s - loss: 3.63

Epoch 58/200
324/327 [============================>.] - ETA: 0s - loss: 3.0287e-04
Epoch 00058: val_loss did not improve from 0.00043
327/327 [==============================] - 2s 5ms/step - loss: 3.0232e-04 - val_loss: 5.7974e-04
Epoch 59/200
323/327 [============================>.] - ETA: 0s - loss: 3.0154e-04
Epoch 00059: val_loss did not improve from 0.00043
327/327 [==============================] - 2s 5ms/step - loss: 3.0028e-04 - val_loss: 5.7738e-04
train_score
in test datsaet, zeros: 7154, non_zeros: 8542
in prediction, zeros: 0, non_zeros: 15696
true_positive: 0, true_negative: 8543, false_positive: 0, false_negative: 7154
nrmse: 0.030745, accuracy: 0.544244, f1_score: -1.000000
{'nrmse': 0.030745480842020785, 'accuracy': 0.5442441230808435, 'f1_score': -1}
val_score
in test datsaet, zeros: 2547, non_zeros: 2637
in prediction, zeros: 0, non_zeros: 5184
true_positive: 0, true_negative: 2638, false_positive: 0, false_negative: 2547
nrmse: 0.204704, accuracy: 0.508775, f1_score:

Epoch 23/200
154/164 [===========================>..] - ETA: 0s - loss: 3.7825e-04
Epoch 00023: val_loss improved from 0.00050 to 0.00047, saving model to /home/vislab/Github/SolarPowerPrediction/results/LSTM-001/feature-11/model-023-0.000472.h5
164/164 [==============================] - 1s 5ms/step - loss: 3.7186e-04 - val_loss: 4.7220e-04
Epoch 24/200
155/164 [===========================>..] - ETA: 0s - loss: 3.7683e-04
Epoch 00024: val_loss improved from 0.00047 to 0.00045, saving model to /home/vislab/Github/SolarPowerPrediction/results/LSTM-001/feature-11/model-024-0.000454.h5
164/164 [==============================] - 1s 5ms/step - loss: 3.7118e-04 - val_loss: 4.5415e-04
Epoch 25/200
155/164 [===========================>..] - ETA: 0s - loss: 3.7312e-04
Epoch 00025: val_loss improved from 0.00045 to 0.00044, saving model to /home/vislab/Github/SolarPowerPrediction/results/LSTM-001/feature-11/model-025-0.000444.h5
164/164 [==============================] - 1s 5ms/step - loss: 3.676

train_score
in test datsaet, zeros: 7154, non_zeros: 8542
in prediction, zeros: 0, non_zeros: 15696
true_positive: 0, true_negative: 8543, false_positive: 0, false_negative: 7154
nrmse: 0.030200, accuracy: 0.544244, f1_score: -1.000000
{'nrmse': 0.030199865277099728, 'accuracy': 0.5442441230808435, 'f1_score': -1}
val_score
in test datsaet, zeros: 2547, non_zeros: 2637
in prediction, zeros: 0, non_zeros: 5184
true_positive: 0, true_negative: 2638, false_positive: 0, false_negative: 2547
nrmse: 0.229170, accuracy: 0.508775, f1_score: -1.000000
{'nrmse': 0.22917031780557254, 'accuracy': 0.5087753134040501, 'f1_score': -1}
test_score
in test datsaet, zeros: 2365, non_zeros: 2819
in prediction, zeros: 0, non_zeros: 5184
true_positive: 0, true_negative: 2820, false_positive: 0, false_negative: 2365
nrmse: 0.219929, accuracy: 0.543877, f1_score: -1.000000
{'nrmse': 0.21992945809985168, 'accuracy': 0.5438765670202508, 'f1_score': -1}
count_params: 865793
Saving /home/vislab/Github/SolarPowerP

In [6]:
first_pred = y_pred_list[0]
first_pred = first_pred.reshape(-1, 1)
first_pred = power_data['scaler'].inverse_transform(first_pred)
first_pred = first_pred.reshape((first_pred.shape[0]))
first_pred.shape

(5184, 1)